### 아이디어
- 첫째로 Counter의 most_common(1)을 사용해서 날먹을 시도하려 했으나, 그 메소드는 $\Omicron(n \log n)$ 의 정렬을 기반으로 작동하는 것을 발견했다. 아마 안될 것이다.
- 관찰1. 구간의 영역이 1칸 확장/축소될 때, 가장 많이 등장하는 수는 많아야 그 변화량이 1밖에 되지 않는다.
  - => 업데이트 한 숫자에 의해 가장 많이 등장하는 수가 바뀐다면, 그 숫자로 바꿔주기만 하면 될 것이다.
- 관찰1 로부터 다음과 같은 방법을 떠올릴 수 있다.
  - 가장 많이 등장하는 수를 $a$ 라고 했을 때, 등장 횟수를 업데이트하면서 최댓값이 갱신되면 $a$ 를 갱신시켜준다. 그렇지 않을경우 그냥 cnt만 1 늘린다.
  - 그러나 이 방법은 $a$ 가 감소할 때의 처리가 곤란하다. $a$ 가 감소했을 때 그 다음으로 많은 수를 찾아야 하는데, 이를 위해서는 $\Omicron(n)$ 의 시간이 필요하다.
    - 최댓값을 갱신할 때 그 이전의 수를 최댓값의 후보로 설정해두는 구현을 생각해봤지만, 이 경우에도 후보값이 계속 감소한다면 또다시 $\Omicron(n)$ 의 시간이 필요하다.
  - 문제를 풀기 위해선 $T(N)$ 의 시간이 $O(\log N)$ 이하여야 한다. 따라서 이 방법은 적합하지 않다.

### 풀이
- 아이디어에 나온 관찰의 확장으로 다음과 같은 list를 생각해볼 수 있다.
  - res = `가장 많이 등장하는 수의 등장 횟수`
  - cnt[i] = `i번째 숫자가 등장한 횟수`
  - cntcnt[y] = `cnt[i] == y` 를 만족하는 y의 개수
- 즉, 현재 가장 많은 수에만 관심을 가지지 않고, 모든 숫자의 등장 횟수를 관리하는 것이다. 대신 그 숫자가 무엇인지는 중요하지 않다.

In [ ]:
import io, os, math, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

MAX = 100000
def sol() :
  N = int(input())
  L = [*map(int, input().split())]

  sqrtN = math.isqrt(N)
  Q = []

  M = int(input())
  for i in range(M):
    s, e = map(int, input().split())
    Q.append((s-1, e, i))
  Q.sort(key=lambda x: (x[0] // sqrtN, x[1]))

  res = 0 #가장 많이 등장한 수의 개수
  cnt = [0] * (MAX + 1) #각 숫자의 등장 횟수
  cntcnt = [0] * (MAX + 1) #cnt[i]가 등장한 횟수
  ans = [0] * M
  def increase(x):
    nonlocal res
    cntcnt[cnt[x]] -= 1 #이전에 cnt[L[i]]가 등장한 횟수를 감소시킨다. 참고로 초기상태의 경우 cnt[0]의 등장횟수가 감소해서 음수가 되는데, A_i가 1~10만이라 아무 상관없다.
    cntcnt[cnt[x] + 1] += 1
    if cnt[x] == res: #이번에 증가시킬 경우 최빈값이 바뀌는 경우이다.
      res += 1
    cnt[x] += 1
  
  def decrease(x) :
    nonlocal res
    cntcnt[cnt[x]] -= 1
    cntcnt[cnt[x] - 1] += 1
    if cnt[x] == res and cntcnt[res] == 0: #감소시킬 값이 최빈값이였고, 감소시켰을 때 최빈값이 아니게 된다면
      res -= 1
    cnt[x] -= 1

  s = e = 0
  for ns, ne, idx in Q:
    while ns < s:
      s -= 1
      increase(L[s])
    while e < ne: 
      increase(L[e])
      e += 1
    while ns > s: 
      decrease(L[s])
      s += 1
    while e > ne: 
      e -= 1
      decrease(L[e])
    
    ans[idx] = res
  
  sys.stdout.write('\n'.join(map(str, ans)))

sol()

- 예제 정답이 이상하다면, 예제 입력을 제대로 넣은건지 다시 검토해보자..